#  Graph RAG using LangChain and Kuzu 

Taken  from https://pypi.org/project/langchain-kuzu/ 

In [34]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks

import builtins

from devtools import debug
from dotenv import load_dotenv

setattr(builtins, "debug", debug)
load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f
!export PYTHONPATH=":./python"
# cSpell: disable

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from pathlib import Path

import kuzu
from ipycytoscape import CytoscapeWidget
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_kuzu.chains.graph_qa.kuzu import KuzuQAChain
from langchain_kuzu.graphs.kuzu_graph import KuzuGraph
from langchain_openai import ChatOpenAI

from python.ai_core.llm import get_llm


In [36]:
DB = "test_db3"
llm = get_llm(llm_id="gpt_4omini_openai")

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

text = "Tim Cook is the CEO of Apple. Apple has its headquarters in California."

if True:  # ne schema
    allowed_nodes = ["Person", "Company", "Location"]
    allowed_relationships = [
        ("Person", "IS_CEO_OF", "Company"),
        ("Company", "HAS_HEADQUARTERS_IN", "Location"),
    ]
    # Define the LLMGraphTransformer
    llm_transformer = LLMGraphTransformer(
        llm=llm,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
    )

    # Convert the given text into graph documents
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    db = kuzu.Database(DB)
    graph = KuzuGraph(db, allow_dangerous_requests=True)

    # Add the graph document to the graph
    graph.add_graph_documents(
        graph_documents,
        include_source=True,
    )
else:
    print("load {DB}")


2025-01-23 18:04:40.956 | INFO     | python.ai_core.llm:get_llm:491 - get LLM:'gpt_4omini_openai'


In [37]:
llm = get_llm(llm_id="gpt_4omini_openai")


2025-01-23 18:04:43.276 | INFO     | python.ai_core.llm:get_llm:491 - get LLM:'gpt_4omini_openai'


In [38]:
db = kuzu.Database(DB)
graph = KuzuGraph(db, allow_dangerous_requests=True)

# Create the KuzuQAChain with verbosity enabled to see the generated Cypher queries
chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.3),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

# Query the graph
queries = [
    "Who is the CEO of Apple?",
    "Where is Apple headquartered?",
]

for query in queries:
    result = chain.invoke(query)
    print(f"Query: {query}\nResult: {result}\n")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:IS_CEO_OF]->(c:Company) WHERE LOWER(c.id) = 'apple' RETURN p
Full Context:
[{'p': {'_id': {'offset': 0, 'table': 1}, '_label': 'Person', 'id': 'Tim Cook', 'type': 'entity'}}]

> Finished chain.
Query: Who is the CEO of Apple?
Result: {'query': 'Who is the CEO of Apple?', 'result': 'Tim Cook is the CEO of Apple.'}



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (c:Company {id: LOWER('Apple')})-[:HAS_HEADQUARTERS_IN]->(l:Location) RETURN l
Full Context:
[]

> Finished chain.
Query: Where is Apple headquartered?
Result: {'query': 'Where is Apple headquartered?', 'result': "I don't know the answer."}



In [39]:
# Create Cytoscape widget
from python.ui_components.cypher_graph_display import get_cytoscape_json, get_cytoscape_style

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_json(get_cytoscape_json(graph))
cyto.set_style(get_cytoscape_style())
# Set layout and style
cyto.set_layout(animate=True)


In [40]:
# Display the graph
cyto

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'animate': True}, cytoscape_style=[{'selector': 'node', 'css…